In [1]:
import numpy as np
import pandas as pd
import os

In [118]:
!pip install altair
!pip install altair vega_datasets
!pip install vega
!pip install altair_viewer
!pip install textwrap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement textwrap (from versions: none)
ERROR: No matching distribution found for textwrap


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls
%cd "/content/drive/MyDrive/Colab Notebooks/skema/data"
!ls

drive  sample_data
/content/drive/MyDrive/Colab Notebooks/skema/data
'1471-2334-3-19 (2) (2).pdf'
 cosmos-and-extractions-jsons-for-3-papers
 data_modeling_covid_italy.json
 data-response-to-covid-19-was-italy-unprepared.json
 data-sars-double.json
 doc_pg_blk_sent_event.xlsx
 modeling_covid_italy.json
 modeling_covid_italy.xlsx
'modelling_doc_event (1).xlsx'
 modelling_doc_event.xlsx
 response-to-covid-19-was-italy-unprepared.json
 response-to-covid-19-was-italy-unprepared.pdf
 response-to-covid-19-was-italy-unprepared.xlsx
 s41591-020-0883-7.pdf
 sarsdouble.json
 sarsdouble.xlsx


In [5]:
path = "/content/drive/MyDrive/Colab Notebooks/skema/data/"
os.path.exists(path)

True

In [6]:
filenames = ["sarsdouble.xlsx", "modeling_covid_italy.xlsx", "response-to-covid-19-was-italy-unprepared.xlsx"]
diff_distance_map = []
for filename in filenames:
  print("\n**********************************************************************************\n")
  print("The annotated extractions file name %s " %(filename))
  df = pd.read_csv(os.path.join(path, filename))
  df['locationContext'] = df['locationContext'].replace({"^'|'$": ""}, regex=True)
  df['temporalContext'] = df['temporalContext'].replace({"^'|'$": ""}, regex=True)

  print("Number of page numbers different from extractions with that of manually annotated are : %d out of %d" %(len(df[df['pg_num'] != df["page_num"]]), df.shape[0]) )
  gp = df.groupby(['locationContext']).count()
  print(gp.reset_index()[['locationContext', 'event_id']])


**********************************************************************************

The annotated extractions file name sarsdouble.xlsx 
Number of page numbers different from extractions with that of manually annotated are : 11 out of 172
                                      locationContext  event_id
0                                             Beijing        17
1                                     China,Hong Kong        15
2                    China,Hong Kong,Beijing,Shanghai         1
3     China,Hong Kong,Mainland China,Beijing,Shanghai         1
4             Europe,Inner Mongolia,Hong Kong,Beijing         1
5   Guangdong Province,Hong Kong,China,Mainland Ch...         2
6   Guangdong Province,Hong Kong,Mainland China,Be...         2
7                           Guangdong,China,Hong Kong         1
8                                           Hong Kong        43
9                              Hong Kong,Amoy Gardens        21
10                                Hong Kong,Guangdong   

In [7]:
print("Distances between annotated page numbers cosmos page numbers: ", len(df[df['pg_num'] != df["page_num"]]))

Distances between annotated page numbers cosmos page numbers:  5


In [66]:
import json
from collections import Counter

filenames = {"sarsdouble":["sarsdouble.json","extractions_sarsdouble.json"], "modeling_covid_italy":["modeling_covid_italy.json","extractions_modeling_covid_italy--COSMOS-data.json"], 
             "response-to-covid-19-was-italy-unprepared":["response-to-covid-19-was-italy-unprepared.json","extractions_response-to-covid-19-was-italy-unprepared--COSMOS-data.json"]}

extractions_path = "cosmos-and-extractions-jsons-for-3-papers"
loc_counter = {}
temporal_counter = {}

for name,filename_pair in filenames.items():
  ann, extr = filename_pair
  print("\n**********************************************************************************\n")
  print("The annotations & extractions for PDF file name %s " %(name))
  
  with open(os.path.join(path,extractions_path, extr ), "r", encoding='UTF-8') as f:
    contents = f.readlines()
    extractions = json.loads(contents[0])

  with open(os.path.join(path, ann ), "r", encoding='UTF-8') as f:
    contents = f.read()
    annotations = json.loads(contents)

  location_contexts = []
  loc_counter[name] = Counter()
  temporal_contexts = []
  temporal_counter[name] = Counter()
  for a in annotations:
    doc_id, page_num, _,event_id, event, locationContext, temporalContext, explanation, pg_num, blk_id, sentence_id, doc_sentence_count = list(a.values())
    #print(locationContext.split(","))
    if locationContext != "":
      location_contexts.extend(locationContext.split(","))
    if temporalContext != "":
      temporal_contexts.extend(temporalContext.split(","))
  print(Counter(location_contexts), Counter(temporal_contexts))
  loc_counter[name] = Counter(location_contexts)
  temporal_counter[name] = Counter(temporal_contexts)


**********************************************************************************

The annotations & extractions for PDF file name sarsdouble 
Counter({'Hong Kong': 128, 'Beijing': 39, 'Inner Mongolia': 39, 'Guangdong': 24, 'Amoy Gardens': 21, 'China': 20, 'Guangdong Province': 7, 'Shanghai': 6, 'Mainland China': 5, 'Europe': 1, 'Paris': 1}) Counter({' 2003': 304, 'March 17': 135, 'May 10': 135, '1983': 21, '1985': 21, '03/17': 10, '03/20': 10, '03/23': 10, '03/26': 10, '03/29': 10, '04/01': 10, '04/04': 10, '04/07': 10, '04/10': 10, '04/13': 10, '04/16': 10, '04/19': 10, '04/22': 10, '04/25': 10, '04/28': 10, '05/01': 10, '05/04': 10, '05/07': 10, '05/10': 10, 'February 21st': 9, 'February 21': 9, '17 March': 8, '10 May': 8, 'November 2002': 5, 'February 22': 4})

**********************************************************************************

The annotations & extractions for PDF file name modeling_covid_italy 
Counter({'Italy': 253, 'N/A': 6, 'Lodi Province': 1, 'Lombardy': 1, 

In [68]:
loc_counter[name].keys()

dict_keys(['Rome', 'Italy', 'Lombardy', 'Veneto', 'Emilia-Romagna', 'Piedmont', 'Italian', 'European countries and the USA', 'Italian regions', 'western European countries', 'Spain', 'northern Italy', 'northern regions'])

In [143]:
import altair as alt
from vega_datasets import data

alt.renderers.enable('colab')
graphs = []
alt.themes.enable('default')
for name in loc_counter.keys():
  this_pdf_locations = loc_counter[name]
  x = list(this_pdf_locations.keys())
  y = list(this_pdf_locations.values())
  df = pd.DataFrame.from_dict(this_pdf_locations, orient="index").reset_index()
  df = df.rename(columns={"index":"Location", 0:"Count"})
  
  bars = alt.Chart(df).mark_bar(size=30).encode(
      y=alt.Y('Location:O', axis=alt.Axis(labelAngle=-45),scale=alt.Scale(zero=False), sort='-x'),
      x=alt.X('Count:Q')
  )

  text = bars.mark_text(
      align='left',
      baseline='bottom', 
      dy = 1,
      dx=10  # Nudges text to right so it doesn't appear on top of the bar
  ).encode(
      text='Count:Q'
  )

  ch = (bars + text).properties(height=550, width=700, title="# Location contexts for %s"%(name))
  
  ch.configure_header(
      titleColor='green',
      titleFontSize=14,
      labelColor='red',
      labelFontSize=14, 
  )
  ch.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
  ).interactive()
  # ch.configure_scale(
  #   bandPaddingOuter=0.3
  # )
  ch.display()
  


alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

In [147]:
import altair as alt
from vega_datasets import data
from textwrap import wrap
alt.renderers.enable('colab')
graphs = []
alt.themes.enable('default')
for name in temporal_counter.keys():
  this_pdf_locations = temporal_counter[name]
  x = list(this_pdf_locations.keys())
  y = list(this_pdf_locations.values())
  df = pd.DataFrame.from_dict(this_pdf_locations, orient="index").reset_index()
  df = df.rename(columns={"index":"Temporal Context", 0:"Count"})
  df['Temporal Context'] = df['Temporal Context'].apply(wrap, args=[30])
  
  bars = alt.Chart(df).mark_bar(size=10).transform_calculate(
    x="split(datum.x, ' ')").encode(
      y=alt.Y('Temporal Context:O', axis=alt.Axis(labelAngle=-45, labelFontSize=9), sort='-x'),
      x=alt.X('Count', )
  )

  text = bars.mark_text(
      align='right',
      baseline='bottom', 
      dy = -2,
      dx=15  # Nudges text to right so it doesn't appear on top of the bar
  ).encode(
      text='Count:Q'
  )

  ch = (bars + text).properties(height=800, width=500, title="# Temporal contexts for %s"%(name))
  
  ch.configure_header(
      titleColor='green',
      titleFontSize=14,
      labelColor='red',
      labelFontSize=14, 
  )
  ch.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
  ).interactive()
  # ch.configure_scale(
  #   bandPaddingOuter=0.3
  # )
  ch.display()
  


alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

In [65]:
df.head(15)

,Location,Count
0,Rome,4
1,Italy,38
2,Lombardy,8
3,Veneto,1
4,Emilia-Romagna,1
5,Piedmont,1
6,Italian,1
7,European countries and the USA,2
8,Italian regions,1
9,western European countries,1


In [37]:
 df.rename(columns={"index":"Location", 0:"Count"})

,Location,Count
0,Rome,4
1,Italy,38
2,Lombardy,8
3,Veneto,1
4,Emilia-Romagna,1
5,Piedmont,1
6,Italian,1
7,European countries and the USA,2
8,Italian regions,1
9,western European countries,1


In [ ]:
df.head()

,Unnamed: 0,doc_id,page_num,para_num,event_id,event,locationContext,temporalContext,explanation,pg_num,blk_id,sentence_id,doc_sentence_count
0,0,1441143227,1,2,E:-1716002397,detection of the first two COVID-19 cases,"Rome,Italy","between 31st January 2020,beginning of June",NaN,1,11,1,8
1,1,1441143227,1,2,E:902720017,excess mortality of +118%,"Lombardy,Italy",two months after the beginning of the first wave,NaN,1,11,4,8
2,2,1441143227,1,2,E:-1163634376,average mortality rate of the period 1 January...,Lombardy,1 January–30 April 2015–2019,NaN,1,11,4,8
3,3,1685655814,2,1,E:378589066,"Livelli Essenziali di Assistenza, LEA",Italy,since the early 1990s,NaN,2,3,6,14
4,4,1685655814,2,1,E:2048573837,development of 21 different regional/provincia...,Italy,since the early 1990s,NaN,2,3,9,14


In [ ]:
df.shape

(172, 12)

In [ ]:
len(df[df['pg_num'] != df["page_num"]])

11

In [ ]:
os.path.exists(os.path.join(path, filename))


True